In [4]:
!pip install arch
!pip install yfinance
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import seaborn as sns
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

     |████████████████████████████████| 808kB 3.4MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=dc97cccd7055eb18dd2f4e63f87e2d8a7deb7694c3c547722e0434222b728b6d
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [5]:
raw_data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = "1994-01-07", end = "2018-01-29", 
                              interval = "1d", group_by = 'ticker', auto_adjust = True, treads = True)

[*********************100%***********************]  4 of 4 completed


In [0]:
df_comp = raw_data.copy()

In [0]:

df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

In [0]:
df_comp = df_comp.iloc[1:]
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

In [0]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

In [0]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

In [11]:
!pip install pmdarima
from pmdarima.arima import auto_arima
model_auto = auto_arima(df.ret_ftse[1:])


Dep. Variable:	y	No. Observations:	5020
Model:	SARIMAX(2, 0, 5)	Log Likelihood	-7885.690
Date:	Fri, 05 Jan 2020	AIC	15789.380
Time:	10:19:03	BIC	15848.070
Sample:	0	HQIC	15809.946
- 5020		
Covariance Type:	opg		
coef	std err	z	P>|z|	[0.025	0.975]
intercept	0.0309	0.024	1.289	0.197	-0.016	0.078
ar.L1	0.1766	0.039	4.544	0.000	0.100	0.253
ar.L2	-0.8128	0.035	-22.984	0.000	-0.882	-0.743
ma.L1	-0.2005	0.038	-5.239	0.000	-0.275	-0.125
ma.L2	0.7654	0.037	20.436	0.000	0.692	0.839
ma.L3	-0.0953	0.012	-8.246	0.000	-0.118	-0.073
ma.L4	0.0112	0.009	1.229	0.219	-0.007	0.029
ma.L5	-0.1113	0.009	-12.960	0.000	-0.128	-0.094
sigma2	1.3550	0.014	94.014	0.000	1.327	1.383
Ljung-Box (Q):	69.64	Jarque-Bera (JB):	6575.67
Prob(Q):	0.00	Prob(JB):	0.00
Heteroskedasticity (H):	2.00	Skew:	-0.18
Prob(H) (two-sided):	0.00	Kurtosis:	8.60


In [12]:
model_auto

ARIMA(maxiter=50, method='lbfgs', order=(4, 0, 5), out_of_sample_size=0,
      scoring='mse', scoring_args=None, seasonal_order=(0, 0, 0, 0),
      start_params=None, suppress_warnings=False, trend=None,
      with_intercept=True)

In [13]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Statespace Model Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                 5019
Model:               SARIMAX(4, 0, 5)   Log Likelihood               -7882.658
Date:                Tue, 05 May 2020   AIC                          15787.316
Time:                        07:43:44   BIC                          15859.047
Sample:                             0   HQIC                         15812.452
                               - 5019                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0309      0.025      1.246      0.213      -0.018       0.080
ar.L1          0.0135      0.082      0.165      0.869      -0.147       0.174
ar.L2         -0.6690      0.077     -8.645      0.000      -0.821      -0.517
ar.L3         -0.1616      0.072     -2.260      0.024      -0.302      -0.021
ar.L4          0.1898      0.074      2.553      0.011       0.044       0.335
ma.L1         -0.0384      0.081     -0.471      0.637      -0.198       0.121
ma.L2          0.6205      0.078      7.933      0.000       0.467       0.774
ma.L3          0.0592      0.069      0.858      0.391      -0.076       0.194
ma.L4         -0.1836      0.073     -2.510      0.012      -0.327      -0.040
ma.L5         -0.1063      0.010    -11.130      0.000      -0.125      -0.088
sigma2         1.3541      0.015     91.128      0.000       1.325       1.383
===================================================================================
Ljung-Box (Q):                       67.77   Jarque-Bera (JB):              6360.08
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               2.00   Skew:                            -0.19
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.50
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [0]:
model_auto = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5,
                       max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2,
                       maxiter = 50, alpha = 0.05, n_jobs = -1, trend = 'ct', information_criterion = 'oob',
                       out_of_sample = int(len(df_comp)*0.2))

In [0]:
plt.figure(figsize=(12,7))
plt.plot(df['Open'], 'green', color='blue', label='Training Data')
plt.plot(test_data.index, predictions, color='green', marker='o', linestyle='dashed', 
         label='Predicted Price')
plt.plot(test_data.index, test_data['Open'], color='red', label='Actual Price')
plt.title('Microsoft Prices Prediction')
plt.xlabel('Dates')
plt.ylabel('Prices')
plt.xticks(np.arange(0,7982, 1300), df['Date'][0:7982:1300])
plt.legend()

![alt text](https://miro.medium.com/max/1400/1*7__Qhrsf8poNFUqIRCsNuQ.png)